Search Agent (Qwen3-0.6B) 重构版说明

目标：基于本地 Qwen3-0.6B 搭建一个最小可用的搜索 Agent Pipeline。

核心流程：
1. 用户提供初始任务 prompt。
2. 模型逐 token 生成并检测工具调用格式。
3. 遇到搜索工具调用 => 执行搜索 => 将搜索结果注入上下文继续生成。
4. 不再触发工具调用且出现终止标记（<eos> / eos token） => 输出最终链接集合。

Notebook 结构：
- 配置与依赖
- 模型加载
- 工具与解析层 (Tool / SearchTool / ToolCallParser)
- AgentPipeline 主循环 (流式 + 工具接入)
- 示例运行 (单函数入口)
- 扩展建议

约定：
- 工具调用格式候选：`<tool_call:search>查询词` / `search: 查询词` / `[SEARCH] 查询词`
- 工具响应结束 token id（暂用推测值）：151666 (</tool_response>)
- 本示例仅做结构演示，未包含安全过滤、重试、并行优化。

这一部分说明所需的依赖与环境前提，提醒读者检查本地模型目录 `./Qwen3-0.6B` 并在缺包时安装 `transformers`、`huggingface_hub`、`requests`、`beautifulsoup4`。

这个单元会动态检查依赖是否安装，并导入后续流程会用到的核心库；如果缺包会抛出清晰的提示便于补齐环境。

In [45]:
# 依赖检查与核心库导入
import importlib
import sys
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Any

REQUIRED_PACKAGES = [
    ("transformers", "transformers"),
    ("torch", "torch"),
    ("requests", "requests"),
    ("beautifulsoup4", "bs4"),
]
missing = []
for pip_name, import_name in REQUIRED_PACKAGES:
    try:
        importlib.import_module(import_name)
    except ModuleNotFoundError:
        missing.append(pip_name)

if missing:
    raise ModuleNotFoundError(
        "缺失依赖: " + ", ".join(missing) +
        "\n请在当前环境中运行: pip install " + " ".join(missing)
    )

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import requests
from bs4 import BeautifulSoup
import re


下面的单元负责加载本地 Qwen3-0.6B 模型与分词器，并根据是否可用 GPU 自动选择 dtype 和 device_map。

In [46]:
# 模型与分词器加载
MODEL_DIR = Path("Qwen3-0.6B")
if not MODEL_DIR.exists():
    raise FileNotFoundError(f"模型目录不存在: {MODEL_DIR.resolve()}")

print(f"加载模型目录: {MODEL_DIR}")
tokenizer = AutoTokenizer.from_pretrained(str(MODEL_DIR), local_files_only=True)

dtype = torch.float16 if torch.cuda.is_available() else torch.float32
load_kwargs = {"local_files_only": True, "torch_dtype": dtype}
if torch.cuda.is_available():
    load_kwargs["device_map"] = "auto"

model = AutoModelForCausalLM.from_pretrained(str(MODEL_DIR), **load_kwargs)
model.eval()
print("模型加载完成，当前设备:", model.device)


加载模型目录: Qwen3-0.6B
模型加载完成，当前设备: cuda:0
模型加载完成，当前设备: cuda:0


这一段用于快速检查 tokenizer 的特殊标记，以及检索包含 "tool" 关键词的 token 以辅助调试工具调用格式。

In [47]:
# Tokenizer 信息检查
print("特殊 token:", tokenizer.special_tokens_map)
print("其他特殊 token:", tokenizer.additional_special_tokens)

matched = []
for token, tid in tokenizer.get_vocab().items():
    if "tool" in token:
        matched.append((tid, token))
    if len(matched) >= 100:
        break
print("包含 'tool' 的 token 示例 (前 100):")
for tid, token in matched:
    print(tid, token)


特殊 token: {'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}
其他特殊 token: ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']
包含 'tool' 的 token 示例 (前 100):
87183 toolbox
44646 -tools
66572 .toolStripButton
15918 tools
45714 /tools
40224 -tool
65894 Ġtoolkit
78458 (toolbar
25942 Ġtoolbar
22785 _tool
23154 .tools
64448 .toolStripSeparator
88265 .toolbox
84904 .toolStripMenuItem
151666 </tool_response>
76265 _tooltip
65695 _toolbar
46902 ĠtoolStrip
25451 Ġtooltip
60646 -tooltip
7375 Ġtools
65539 toolStrip
51281 uptools
75027 /tool
21268 .tool
3631

后续工具实现依赖一些常量，这里集中定义工具结束 token、搜索模式、HTTP 头等基础配置，保持全局可见。

In [48]:
# 工具与解析层：基础配置
TOOL_END_TOKEN_ID = 151666  # 推测为 </tool_response>
SEARCH_PATTERNS = [
    r"search:\s*(.*)",       # 兼容旧格式 "search: query"
    r"\[TOOL_CALL\]\s*\n\s*search:\s*(.*)", # 兼容旧多行格式
    r"<tool_call:search>(.*?)$",  # 兼容旧XML样式
    r"\[SEARCH\]\s+(.*)$",      # 兼容旧标签
]
# 主要使用 <search>...</search> 新标签；旧模式保留以便混合 prompt 测试
HEADERS = {"User-Agent": "Mozilla/5.0 (SearchAgent Prototype)"}
MAX_LINKS_DEFAULT = 6

紧接着的代码定义抽象工具基类和抓取 jina.ai 搜索结果的 `SearchTool`，并处理异常返回结构化信息。

In [49]:
# 工具实现：SearchTool
class Tool:
    name: str

    def run(self, *args, **kwargs) -> Dict[str, Any]:
        raise NotImplementedError


class SearchTool(Tool):
    name = "search"

    def run(self, query: str, max_links: int = MAX_LINKS_DEFAULT) -> Dict[str, Any]:
        url = f"https://www.jina.ai/search?q={requests.utils.quote(query)}"
        try:
            resp = requests.get(url, headers=HEADERS, timeout=15)
            resp.raise_for_status()
        except Exception as exc:
            return {"query": query, "links": [], "error": str(exc)}

        soup = BeautifulSoup(resp.text, "html.parser")
        links: List[str] = []
        for anchor in soup.select("a[href]"):
            href = anchor.get("href")
            if href and href.startswith("http") and "jina.ai" not in href:
                links.append(href)
            if len(links) >= max_links:
                break
        return {"query": query, "links": links}


search_tool = SearchTool()


为了抽取模型输出中的工具调用，这里实现 `ToolCallParser`，兼容简单正则与预留的 JSON 包装格式。

In [51]:
# 工具调用解析器
class ToolCallParser:
    def find_search(self, text: str):
        for pattern in SEARCH_PATTERNS:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                query = match.group(1).strip()
                query = re.split(r"</tool_response>|<eos>|\n", query)[0].strip()
                if query:
                    return query
        return None

    def find_json_tool(self, text: str):
        # 预留 JSON 工具调用格式: <tool_call>{"name":"search", ...}</tool_call>
        match = re.search(r"<tool_call>\s*(\{.*?\})\s*</tool_call>", text, re.DOTALL)
        if not match:
            return None
        raw_json = match.group(1)
        try:
            import json
            return json.loads(raw_json)
        except Exception:
            return None


parser = ToolCallParser()


在进入主循环前，需要一个统一的对话模板封装，这样模型在没有原生 chat 模板时也能 fallback 到简单的 role:content 拼接。

In [52]:
# Chat 模板包装
def apply_chat_template(messages):
    if hasattr(tokenizer, "apply_chat_template"):
        try:
            return tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
            )
        except Exception:
            pass
    lines = [f"{m['role']}: {m['content']}" for m in messages]
    lines.append("assistant:")
    return "\n".join(lines)

接下来的 Pipeline 单元负责逐 token 流式生成、检测工具调用、执行搜索并把结果拼回消息列表，是整个 Agent 的核心逻辑。

系统提示（System Prompt）

下面新增一个系统级指导信息，要求模型在需要检索时使用形如 <search>关键词</search> 的标签包围搜索词。生成过程中一旦形成闭合标签即触发搜索，将搜索结果与原始关键词回注入上下文继续回答。

In [53]:
# 系统提示定义
GLOBAL_SYSTEM_PROMPT = (
    "你是一个搜索增强助手。判断需要外部信息时，请生成 <search>关键词</search> 标签。"
    " 标签内只放原始搜索关键词，避免多句。形成 </search> 闭合后会自动检索 Jina.ai。"
    " 系统会将检索到的若干链接与原始关键词回传，你需要利用这些结果综合回答。"
    " 如果不需要搜索就直接正常回答并结束。不要虚构搜索结果。"
)

In [54]:
# AgentPipeline 定义
@dataclass
class GenerationResult:
    final_text: str
    search_queries: List[str]
    links: List[str]


class AgentPipeline:
    def __init__(self, model, tokenizer, tool: Tool, parser: ToolCallParser):
        self.model = model
        self.tokenizer = tokenizer
        self.tool = tool
        self.parser = parser

    def _prepare_inputs(self, messages: List[Dict[str, str]]):
        prompt = apply_chat_template(messages)
        if verbose_g: # 使用全局变量来控制打印
            print("--- PROMPT START ---")
            print(prompt)
            print("--- PROMPT END ---\n")
        encoded = self.tokenizer([prompt], return_tensors="pt")
        return (
            encoded["input_ids"].to(self.model.device),
            encoded["attention_mask"].to(self.model.device),
        )

    def _append_message(self, messages: List[Dict[str, str]], role: str, content: str):
        messages.append({"role": role, "content": content})

    def _dedup(self, items: List[str]) -> List[str]:
        seen = set()
        ordered = []
        for item in items:
            if item not in seen:
                seen.add(item)
                ordered.append(item)
        return ordered

    def stream(self, messages: List[Dict[str, str]], *, max_steps: int = 96, verbose: bool = True) -> GenerationResult:
        global verbose_g
        verbose_g = verbose
        
        input_ids, attention_mask = self._prepare_inputs(messages)
        generated: List[int] = []
        cache = None
        collected_links: List[str] = []
        search_queries: List[str] = []
        processed_search_segments: List[str] = []  # 已处理的<search>内容，防重复

        with torch.inference_mode():
            for step in range(max_steps):
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    use_cache=True,
                    past_key_values=cache,
                )
                logits = outputs.logits[:, -1, :]
                cache = outputs.past_key_values

                next_token = torch.argmax(logits, dim=-1)
                token_id = next_token.item()
                generated.append(token_id)

                # 续写输入
                input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=1)
                attention_mask = torch.cat([
                    attention_mask,
                    torch.ones_like(next_token).unsqueeze(0),
                ], dim=1)

                token_text = self.tokenizer.decode([token_id], skip_special_tokens=False)
                full_text = self.tokenizer.decode(generated, skip_special_tokens=False)
                if verbose:
                    print(f"Step {step:02d} | id={token_id} | {repr(token_text)}")

                # 终止条件
                if token_id == self.tokenizer.eos_token_id or "<eos>" in full_text:
                    if verbose:
                        print("[终止] 遇到 EOS 标记，停止生成。")
                    break

                # 旧格式兼容：正则/JSON 检测
                json_call = self.parser.find_json_tool(full_text)
                if json_call and isinstance(json_call, dict):
                    name = json_call.get("name")
                    args = json_call.get("args", {})
                    if verbose:
                        print(f"\n[JSON 工具触发] name={name} args={args}")
                    if name == self.tool.name and "query" in args:
                        query = args["query"]
                        search_queries.append(query)
                        result = self.tool.run(query)
                        links = result.get("links", [])
                        collected_links.extend(links)
                        feedback = "搜索结果:\n" + "\n".join(links or ["(无结果)"])
                        self._append_message(messages, "assistant", full_text)
                        self._append_message(messages, "user", f"关键词: {query}\n" + feedback + "\n请继续整合。")
                        input_ids, attention_mask = self._prepare_inputs(messages)
                        generated.clear()
                        cache = None
                        continue

                # 兼容旧正则 search: / [SEARCH]
                legacy_query = self.parser.find_search(full_text)
                if legacy_query and legacy_query not in processed_search_segments:
                    if verbose:
                        print(f"\n[搜索触发(旧格式)] query={legacy_query}")
                    processed_search_segments.append(legacy_query)
                    search_queries.append(legacy_query)
                    result = self.tool.run(legacy_query)
                    links = result.get("links", [])
                    collected_links.extend(links)
                    feedback = "搜索结果:\n" + "\n".join(links or ["(无结果)"])
                    self._append_message(messages, "assistant", full_text)
                    self._append_message(messages, "user", f"关键词: {legacy_query}\n" + feedback + "\n请继续整合。")
                    input_ids, attention_mask = self._prepare_inputs(messages)
                    generated.clear()
                    cache = None
                    continue

                # 新格式：<search>...</search>
                # 查找所有闭合标签
                search_matches = re.findall(r"<search>(.*?)</search>", full_text, flags=re.DOTALL)
                for match in search_matches:
                    cleaned_query = match.strip().replace("\n", " ")
                    if cleaned_query and cleaned_query not in processed_search_segments:
                        if verbose:
                            print(f"\n[搜索触发(<search>标签)] query={cleaned_query}")
                        processed_search_segments.append(cleaned_query)
                        search_queries.append(cleaned_query)
                        result = self.tool.run(cleaned_query)
                        links = result.get("links", [])
                        collected_links.extend(links)
                        feedback = "搜索结果:\n" + "\n".join(links or ["(无结果)"])
                        # 将当前生成内容作为assistant消息，再追加用户反馈用于下一轮生成
                        self._append_message(messages, "assistant", full_text)
                        self._append_message(
                            messages,
                            "user",
                            f"关键词: {cleaned_query}\n" + feedback + "\n请结合结果继续回答。"
                        )
                        input_ids, attention_mask = self._prepare_inputs(messages)
                        generated.clear()
                        cache = None
                        break  # 本步只处理一个新触发，重新编码后继续循环

        final_text = self.tokenizer.decode(generated, skip_special_tokens=False)
        return GenerationResult(
            final_text=final_text,
            search_queries=search_queries,
            links=self._dedup(collected_links),
        )

为了方便外部调用，下面会构造 `run_search_agent` 包装函数，复用已经初始化好的 Pipeline 并打印关键信息。

In [55]:
# 统一入口函数
pipeline = AgentPipeline(model=model, tokenizer=tokenizer, tool=search_tool, parser=parser)

# 全局变量用于跨单元输出 SUMMARY
global_last_query = None
global_last_cleaned = None

def _clean_assistant_output(raw: str) -> str:
    import re
    cleaned = re.sub(r'<think>.*?</think>', '', raw, flags=re.DOTALL)
    cleaned = cleaned.replace('<|im_end|>', '').replace('<eos>', '')
    cleaned = re.sub(r'\n{3,}', '\n\n', cleaned).strip()
    cleaned = cleaned.replace('</think>', '')
    return cleaned

def run_search_agent(query: str, *, max_steps: int = 72, verbose: bool = True) -> GenerationResult:
    global global_last_query, global_last_cleaned
    messages = [
        {"role": "system", "content": GLOBAL_SYSTEM_PROMPT},
        {"role": "user", "content": query},
    ]
    result = pipeline.stream(messages, max_steps=max_steps, verbose=verbose)
    raw_final = result.final_text
    assistant_output = _clean_assistant_output(raw_final)

    # 保留旧格式块
    print("\n=== 提取到的查询 ===", result.search_queries)
    print("=== 去重后链接 ===")
    if result.links:
        for i,l in enumerate(result.links,1):
            print(i,l)
    else:
        print("(无)")
    print("\n=== 模型输出片段 (原始前400) ===")
    print(raw_final[:400])
    print("\n==============================")

    global_last_query = query
    global_last_cleaned = assistant_output if assistant_output else '(空)'
    return result

最后的示例单元提供一个默认 prompt，方便手动触发一次搜索调用进行调试；默认注释掉实际调用以免误运行。

In [56]:
# 示例调用 1: 简单查询
simple_query = "hello"
print("--- Running Simple Query ---")
simple_result = run_search_agent(simple_query, verbose=True)
print("\n" + "="*30 + "\n")

--- Running Simple Query ---
--- PROMPT START ---
<|im_start|>system
你是一个搜索增强助手。判断需要外部信息时，请生成 <search>关键词</search> 标签。 标签内只放原始搜索关键词，避免多句。形成 </search> 闭合后会自动检索 Jina.ai。 系统会将检索到的若干链接与原始关键词回传，你需要利用这些结果综合回答。 如果不需要搜索就直接正常回答并结束。不要虚构搜索结果。<|im_end|>
<|im_start|>user
hello<|im_end|>
<|im_start|>assistant

--- PROMPT END ---

Step 00 | id=151667 | '<think>'
Step 01 | id=198 | '\n'
Step 02 | id=23811 | ' hello'
Step 01 | id=198 | '\n'
Step 02 | id=23811 | ' hello'
Step 03 | id=27641 | '密'
Step 04 | id=58143 | ' 和'
Step 03 | id=27641 | '密'
Step 04 | id=58143 | ' 和'
Step 05 | id=198 | '\n'
Step 06 | id=151668 | '</think>'
Step 05 | id=198 | '\n'
Step 06 | id=151668 | '</think>'
Step 07 | id=46306 | '完'
Step 08 | id=105470 | '相关的'
Step 07 | id=46306 | '完'
Step 08 | id=105470 | '相关的'
Step 09 | id=18947 | '个'
Step 10 | id=16744 | '文'
Step 09 | id=18947 | '个'
Step 10 | id=16744 | '文'
Step 11 | id=14224 | '件'
Step 12 | id=104689 | '不需要'
Step 11 | id=14224 | '件'
Step 12 | id=104689 | '不需要'
Step 13 | id=9

简单 query 的输入输出总结

In [57]:
# SUMMARY 输出（简单查询）
print("SUMMARY\n")
print(f"query:{simple_query}\n")
print(f"response:{_clean_assistant_output(simple_result.final_text) if simple_result else '(空)'}\n")

SUMMARY

query:hello

response:完相关的个文件不需要词
如果，

，
是回答回答一下 

ال于

1️
  ี
。我个求 �ال·
问答
.
结果多取
  原始



In [58]:
# 示例调用 2: 复杂查询 (带<search>标签演示)
complex_query = """
给我一些近期开源中文多模态项目。
<search>开源 中文 多模态 项目</search>
""".strip()
print("--- Running Complex Query ---")
complex_result = run_search_agent(complex_query, verbose=True)

--- Running Complex Query ---
--- PROMPT START ---
<|im_start|>system
你是一个搜索增强助手。判断需要外部信息时，请生成 <search>关键词</search> 标签。 标签内只放原始搜索关键词，避免多句。形成 </search> 闭合后会自动检索 Jina.ai。 系统会将检索到的若干链接与原始关键词回传，你需要利用这些结果综合回答。 如果不需要搜索就直接正常回答并结束。不要虚构搜索结果。<|im_end|>
<|im_start|>user
给我一些近期开源中文多模态项目。
<search>开源 中文 多模态 项目</search><|im_end|>
<|im_start|>assistant

--- PROMPT END ---

Step 00 | id=151667 | '<think>'
Step 01 | id=198 | '\n'
Step 02 | id=29258 | '重'
Step 01 | id=198 | '\n'
Step 02 | id=29258 | '重'
Step 03 | id=13072 | '名'
Step 04 | id=198 | '\n'
Step 03 | id=13072 | '名'
Step 04 | id=198 | '\n'
Step 05 | id=59258 | '近'
Step 06 | id=78973 | '搜索'
Step 05 | id=59258 | '近'
Step 06 | id=78973 | '搜索'
Step 07 | id=5373 | '、'
Step 08 | id=198 | '\n'
Step 07 | id=5373 | '、'
Step 08 | id=198 | '\n'
Step 09 | id=198 | '\n'
Step 10 | id=198 | '\n'
Step 09 | id=198 | '\n'
Step 10 | id=198 | '\n'
Step 11 | id=198 | '\n'
Step 12 | id=198 | '\n'
Step 11 | id=198 | '\n'
Step 12 | id=198 | '\n'
Step 13 | id=198 | '\n

复杂 query 的输入输出总结

In [59]:
# SUMMARY 输出（复杂查询）
print("SUMMARY\n")
print(f"query:{complex_query}\n")
print(f"response:{_clean_assistant_output(complex_result.final_text) if complex_result else '(空)'}\n")

SUMMARY

query:给我一些近期开源中文多模态项目。
<search>开源 中文 多模态 项目</search>

response:<think>
重名
近搜索、

文的是
</:不需要

 <
，回答和
没有
 ี
 �

</
很
作用
多

表明一个
 �中如果

.
:个学要一个的要

了要



# 下一步可以优化的内容
- 结构化 JSON 工具调用：`<tool_call>{"name":"search","args":{"query":"..."}}</tool_call>`
- 重试与速率限制，避免过频访问
- 多搜索源聚合与去重合并
- 搜索结果摘要与相关性评分
- 采样策略：top-k / nucleus 代替 argmax
- 对话与搜索缓存（LRU / SQLite）
- 域名白名单 / 恶意内容过滤